In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.7 MB/s eta 0:00:00


In [2]:
# 필요한 모듈 import
import torch
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2LMHeadModel
# from transformers import AutoTokenizer, TFGPT2LMHeadModel
import pandas as pd
import tqdm
import urllib.request

In [3]:
# kogpt-skt 불러오기
# 기존 코드의 AutoTokenizer와 TFGPT2LMHeadModel를 사용할 경우 token id가 실습과 다른 현상이 발생
# 다른 방법으로 import 해준다
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>',\
                                                     pad_token='<pad>', mask_token='<mask>')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.6.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [4]:
print(tokenizer.bos_token_id) # 문장의 시작을 의미하는 token
print(tokenizer.eos_token_id) # 문장의 끝
print(tokenizer.pad_token_id) # 어느 부분이 padding 되었는지 알기 위함 padding 토큰
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

print(tokenizer.encode('<usr>'))

1
1
3
----------
</s>
<usr>
<pad>
<sys>
[2]


In [5]:
# data load
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [6]:
len(train_data)

11823

In [7]:
# out of memory가 말이 됩니까 코렙 양반 이거 pro인데
batch_size = 16

In [8]:
# 문장의 시작 토큰 + 문장(토큰화) + 문장의 끝 토큰
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)
    yield bos_token + sent + eos_token

In [9]:
''' 수정 전 코드
def get_chat_data(): #generator함수 사용하여 dataset 구성
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id] #문장 시작 토큰
    eos_token = [tokenizer.eos_token_id] #문장 끝 토큰
    # user token, 현재 이것이 실습과 다르게 입력으로 들어가고 있지 않음, 추가로 해줌
    # 특정 토큰 id가 있을텐데 hugging face에서 찾아봐야할 듯
    usr_token = [2]
    sys_token = [4]
    q = tokenizer.encode('' + question)
    a = tokenizer.encode('' + answer)
    yield bos_token + usr_token + q + sys_token + a + eos_token'''

" 수정 전 코드\ndef get_chat_data(): #generator함수 사용하여 dataset 구성\n  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):\n    bos_token = [tokenizer.bos_token_id] #문장 시작 토큰\n    eos_token = [tokenizer.eos_token_id] #문장 끝 토큰\n    # user token, 현재 이것이 실습과 다르게 입력으로 들어가고 있지 않음, 추가로 해줌\n    # 특정 토큰 id가 있을텐데 hugging face에서 찾아봐야할 듯\n    usr_token = [2]\n    sys_token = [4]\n    q = tokenizer.encode('' + question)\n    a = tokenizer.encode('' + answer)\n    yield bos_token + usr_token + q + sys_token + a + eos_token"

In [10]:
# from_generator: 데이터셋 제작 파이프라인, 대용량 train data 데이터 처리에 용이
# 효율적인 메모리 사용 방식을 통해 dataset을 생성한다.
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32) # 토큰화 과정은 함수에 있음

In [11]:
# padding
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [12]:
# 1은 bos, eos token
# 3은 padding token
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
   7098 25856     1     3     3     3     3     3     3]
 [    1     2  9020  8263  7497 10192 11615  8210  8006     4 12422  8711
   9535  7483 12521     1     3     3     3     3     3]
 [    1     2  9085  7597   395  8149 10624  7397 24224 13358  7182     4
  12079  8135 16899  9677  8234   389     1     3     3]
 [    1     2  9085  7597   395  8149  9465 10624  7397 24224 13358  7182
      4 12079  8135 16899  9677  8234   389     1     3]
 [    1     2  9943   422   418  9327  8702  7098     4  9847 16912 18328
   8671  7415  8263  8234   389     1     3     3     3]
 [    1     2  9815   410 21249 10174  6824  8210  8006     4  9427 11056
  11594 10137 10556  9266  8711 25856     1     3     3]
 [    1     2  9815   410 21249  9183  7249     4  9427 11056 11594 10137
  10556  9266  8711 25856     1     3     3     3     3]
 [    1     2  9815 37655  9622  8619 10401  9183  9328   216     4  944

In [13]:
# 다시 문장으로 변환해보자
print(tokenizer.decode(batch[0]))

# 토큰화 된건 어떤 모양일까?
print(batch[0])

print(tokenizer.encode('</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'))

</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad>
tf.Tensor(
[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
  7098 25856     1     3     3     3     3     3     3], shape=(21,), dtype=int32)
[1, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [14]:
# optimizer 정의
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [15]:
steps = len(train_data) // batch_size + 1
print(steps)

739


In [17]:
# cuda delete cache, out of memory 방지
torch.cuda.empty_cache()

학습은 되거든요? 돌아가는데 out of memory가 계속 떠요
서버 지원 좀 흑흑 저 심지어 프론데 왜 이게 안 돌아가요?\
진짜 코랩 개 너무함 흑

In [18]:
# 학습해보자!
EPOCHS = 1

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.notebook.tqdm(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)

      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/739 [00:00<?, ?it/s]

InvalidArgumentError: ignored

In [ ]:
text = '오늘도 좋은 하루!'

sent = '<usr>' + text + '<sys>'

input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])

In [ ]:
# 생성해보기
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence.split('<sys> ')[1].replace('</s>', ''))

In [ ]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].numpy().tolist())

In [ ]:
# 응답 생성하기
def return_answer_by_chatbot(user_text):
  sent = '' + user_text + ''
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split(' ')[1].replace('', '')
  return chatbot_response

In [ ]:
return_answer_by_chatbot('안녕! 반가워~')

In [ ]:
return_answer_by_chatbot('배고파')

In [ ]:
return_answer_by_chatbot('공부하기 싫어')